## IMPORTANT
Make sure all core_functions and configs are loaded before running this stand alone version

In [ ]:
%pip install matplotlib folium

In [ ]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_ADHOC_BILLING')
# Query to extract relevant data from BigQuery
query = """
WITH geo_changes AS (
  SELECT
    ip_address,
    zip_code,
    neustar_country as country,
    neustar_state as start,
    timestamp,
    LAG(zip_code) OVER (PARTITION BY ip_address ORDER BY timestamp) AS prev_zip_code,
    LAG(timestamp) OVER (PARTITION BY ip_address ORDER BY timestamp) AS prev_timestamp
  FROM `bigquery-sandbox-393916.looker.pageviews`
  WHERE neustar_country in ('us', 'ca')
)
SELECT
  ip_address,
  zip_code,
  prev_zip_code,
  timestamp,
  prev_timestamp,
  TIMESTAMP_DIFF(timestamp, prev_timestamp, DAY) AS stability_duration_days
FROM geo_changes
WHERE prev_timestamp IS NOT NULL
"""

# Run the query and load results into a Pandas DataFrame
print("Querying BigQuery...")

df = core_functions.fetch_gbq_data(query, bigquery_client)
# query_job = client.query(query)
# results = query_job.result()
# df = results.to_dataframe()

# Calculate metrics
print("Calculating metrics...")

# Filter for rows where the zip code changed
df['has_changed'] = df['zip_code'] != df['prev_zip_code']
changes = df[df['has_changed']]

# Calculate stability metrics
stability_metrics = changes.groupby('ip_address').agg(
    avg_stability_duration_days=('stability_duration_days', 'mean'),
    max_stability_duration_days=('stability_duration_days', 'max'),
    min_stability_duration_days=('stability_duration_days', 'min'),
    change_count=('stability_duration_days', 'count')
).reset_index()

# Calculate overall statistics
overall_avg_stability = stability_metrics['avg_stability_duration_days'].mean()
overall_max_stability = stability_metrics['max_stability_duration_days'].max()
overall_min_stability = stability_metrics['min_stability_duration_days'].min()

# Print insights
print("\n--- Insights ---")
print(f"Overall Average Stability Duration (days): {overall_avg_stability:.2f}")
print(f"Overall Max Stability Duration (days): {overall_max_stability}")
print(f"Overall Min Stability Duration (days): {overall_min_stability}")

# Suggest a cache age based on overall average stability
recommended_cache_age = int(overall_avg_stability * 0.8)  # Set to 80% of the average stability
print(f"\nRecommended Cache Age (days): {recommended_cache_age}")

# Save stability metrics to CSV for further analysis
stability_metrics.to_csv("ip_stability_metrics.csv", index=False)
print("\nStability metrics saved to 'ip_stability_metrics.csv'.")

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
map_geo_sql = f"""
    SELECT geo_location as zip_code, geo_city as city, geo_country as country, geo_state as state,
geo_latitude as latitude, geo_longitude as longitude

from `next90-core-applications.next90_analytics.geos` WHERE geo_type = 'zip' 
AND geo_country in ('United States', 'Canada')
"""
city_reference = core_functions.fix_df_dtypes(core_functions.fetch_gbq_data(query=map_geo_sql, bigquery_client=n90_bigquery_client))

city_reference.head()
city_reference.loc[city_reference['country'] != 'United States']
changes['zip_code'] = changes['zip_code'].apply(lambda x: str(x).upper().replace(' ', ''))


In [ ]:
stability_metrics_2 = changes.groupby(['ip_address', 'zip_code']).agg(
    avg_stability_duration_days=('stability_duration_days', 'mean'),
    max_stability_duration_days=('stability_duration_days', 'max'),
    min_stability_duration_days=('stability_duration_days', 'min'),
    change_count=('stability_duration_days', 'count')
).reset_index()

stability_metrics_2['zip_code'] = stability_metrics_2['zip_code'].apply(lambda x: str(x).upper().replace(' ', ''))

In [ ]:
merged_df = pd.merge(stability_metrics_2, city_reference, on='zip_code', how='left')

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from matplotlib import cm

# Merge stability metrics with city reference DataFrame
merged_df = pd.merge(stability_metrics_2, city_reference, on='zip_code', how='left')

# Group by city and calculate volatility metrics
city_volatility = merged_df.groupby(['city', 'country', 'latitude', 'longitude']).agg(
    avg_stability_duration=('avg_stability_duration_days', 'mean'),
    total_ip_count=('ip_address', 'count'),
    total_change_count=('change_count', 'sum')
).reset_index()

# Calculate recommended cache duration
city_volatility['recommended_cache_duration'] = (city_volatility['avg_stability_duration'] * 0.8).round()

# Normalize cache durations for color gradient
max_cache_duration = city_volatility['recommended_cache_duration'].max()
min_cache_duration = city_volatility['recommended_cache_duration'].min()
norm = cm.colors.Normalize(vmin=min_cache_duration, vmax=max_cache_duration)
color_map = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Create a Folium map centered on North America
map_center = [40.0, -100.0]
m = folium.Map(location=map_center, zoom_start=4, tiles="CartoDB positron")

# Add cities to the map
for _, row in city_volatility.iterrows():
    color = color_map.to_rgba(row['recommended_cache_duration'])
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        color=None,
        fill=True,
        fill_color=cm.colors.rgb2hex(color[:3]),
        fill_opacity=0.7,
        popup=folium.Popup(
            f"<b>City:</b> {row['city']}<br>"
            f"<b>Country:</b> {row['country']}<br>"
            f"<b>Cache Duration (days):</b> {row['recommended_cache_duration']}",
            max_width=300
        )
    ).add_to(m)

# Save the map to an HTML file and display it
map_file = "city_ip_cache_map.html"
m.save(map_file)
print(f"Map saved to {map_file}")


In [ ]:
# Group by state and calculate volatility metrics
state_volatility = merged_df.groupby(['state', 'country', 'latitude', 'longitude']).agg(
    avg_stability_duration=('avg_stability_duration_days', 'mean'),
    total_ip_count=('ip_address', 'count'),
    total_change_count=('change_count', 'sum')
).reset_index()

In [ ]:

# Calculate recommended cache duration as 80% of the average stability duration
state_volatility['recommended_cache_duration'] = (state_volatility['avg_stability_duration'] * 0.8).round()
state_ip_summary = merged_df.groupby(['state', 'country']).agg(
    avg_stability_duration=('avg_stability_duration_days', 'mean'),
    total_ip_count=('ip_address', 'count'),
    total_change_count=('change_count', 'sum')
).reset_index()

state_ip_summary['recommended_cache_duration'] = (state_ip_summary['avg_stability_duration'] * 0.8).round()
state_ip_summary.head()
state_reference = state_ip_summary.copy()

# Load GeoJSON for US states and Canadian provinces
with open("geo-json/us-states.geo.json", "r") as us_file, open("geo-json/canada_provinces.geo.json", "r") as ca_file:
    us_geojson = json.load(us_file)
    ca_geojson = json.load(ca_file)

# Combine GeoJSONs
geojson_data = {"type": "FeatureCollection", "features": us_geojson["features"] + ca_geojson["features"]}

# Add average stability duration and cache duration to GeoJSON properties
for feature in geojson_data["features"]:
    state_name = feature["properties"]["name"]
    match = state_reference[state_reference["state"] == state_name]
    if not match.empty:
        feature["properties"]["avg_stability_duration"] = match.iloc[0]["avg_stability_duration"]
        feature["properties"]["recommended_cache_duration"] = match.iloc[0]["recommended_cache_duration"]
    else:
        feature["properties"]["avg_stability_duration"] = "No data"
        feature["properties"]["recommended_cache_duration"] = "No data"

# Normalize cache durations for color gradient
max_cache_duration = state_reference['recommended_cache_duration'].max()
min_cache_duration = state_reference['recommended_cache_duration'].min()
norm = cm.colors.Normalize(vmin=min_cache_duration, vmax=max_cache_duration)
color_map = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Function to get color for a state/province
def get_color(state_name):
    match = state_reference[state_reference["state"] == state_name]
    if not match.empty:
        cache_duration = match.iloc[0]["recommended_cache_duration"]
        return cm.colors.rgb2hex(color_map.to_rgba(cache_duration)[:3])
    return "#d3d3d3"  # Default gray for states not in the data

# Create a Folium map centered on North America
map_center = [50.0, -100.0]
m = folium.Map(location=map_center, zoom_start=4, tiles="CartoDB positron")

# Add GeoJSON to the map
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        "fillColor": get_color(feature["properties"]["name"]),
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["name", "avg_stability_duration", "recommended_cache_duration"],
        aliases=["State/Province:", "Avg Stability Duration (days):", "Recommended Cache Duration (days):"],
        localize=True
    )
).add_to(m)

# Add a legend
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 140px; 
            background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
            padding: 10px;">
    <b>Cache Duration (days)</b><br>
    <i style="background: #ffffb2; width: 20px; height: 20px; float: left; margin-right: 5px;"></i> Low<br>
    <i style="background: #fecc5c; width: 20px; height: 20px; float: left; margin-right: 5px;"></i> Medium<br>
    <i style="background: #fd8d3c; width: 20px; height: 20px; float: left; margin-right: 5px;"></i> High<br>
    <i style="background: #e31a1c; width: 20px; height: 20px; float: left; margin-right: 5px;"></i> Very High<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file and display it
map_file = "state_province_ip_cache_map.html"
m.save(map_file)
print(f"Map saved to {map_file}")
state_reference.to_csv('state_ip_summary.csv', index=False)